# Renewable energy sources optimization

### Problem formulation

We need to optimize the distribution of the energy load from multiple sources to different consumers.
We need to to minimize the overall cost, while satisfying demand constraints.

### Parameters

- **Energy Sources**: Two sources (e.g., solar and wind), each with different capacities and costs.
- **Consumers**: Two consumers, each with a specific energy demand.
- **Objective**: Minimize the total cost of energy distribution while meeting all consumer demands without exceeding the source capacities.
- **Tools**: Linear programming, Quadratic Programming

- **Sources parameters**:

    | Source            | Capacity (kW) | Cost (€/kW) |
    |-------------------|---------------|-------------|
    | **I** (eg. Solar) | 100           | 0.15        |                    
    | **J**  (eg. Wind) | 150           | 0.20        |                   


- **Consumer Demands**:
    - Consumer A = 90 kW
    - Consumer B = 120 kW

### Formulation

- **Variables**:
    - $x_{i,a}$: Amount of energy transferred from source $i$ to consumer $a$.
- **Objective**
    We can define a linear cost, to be minimized:
    $$ \text{Cost} = cost_I \times (x_{I,A} + x_{I,B}) + cost_J \times(x_{J,A} + x_{J,B}) $$
- **Constraints**
  At any time these constraint must be met:
    1. Total energy supplied to each consumer meets their demand.
        - Demand of Consumer A: $x_{I,A} + x_{J,A} = 90$
        - Demand of Consumer B: $x_{I,B} + x_{J,B} = 120$
    2. The total energy taken from each source does not exceed its capacity.
        - Solar capacity: $x_{I,A} + x_{I,B} \leq 100$
        - Wind capacity: $x_{J,A} + x_{J,B} \leq 150$
    3. All energies must be positive
       - $x_{i,a} \geq 0$ for any $(i, a)$
         

In [1]:
# GLOBAL VARIABLES

In [57]:
from optimization.core.renewables import EnergySource, Consumer, EnergyDistribution

# Create EnergySource instances
solar = EnergySource("Solar", 100, 0.10)
wind = EnergySource("Wind", 150, 0.05)

# Create Consumer instances
consumer_a = Consumer("A", 5)
consumer_b = Consumer("B", 10)
consumer_c = Consumer("C", 20)

# Create the energy distribution system
system = EnergyDistribution()

system.add_source(solar)
system.add_source(wind)

system.add_consumer(consumer_a)
system.add_consumer(consumer_b)
system.add_consumer(consumer_c)

In [58]:
system.consumers

{'A': {'demand': 5}, 'B': {'demand': 10}, 'C': {'demand': 20}}

In [59]:
def cost_function_array(x_array):
    
    # constraint on conusmer_a
    assert x_array[0, :].sum() == consumer_a.demand

    # constraint on conusmer_b
    assert x_array[1, :].sum() == consumer_b.demand

    # constraint on conusmer_c
    assert x_array[2, :].sum() == consumer_c.demand

    # constraint on source_i
    assert x_array[:, 0].sum() <= solar.capacity

    # constraint on source_j
    assert x_array[:, 1].sum() <= wind.capacity
    
    return solar.cost_per_unit * x_array[:, 0].sum() + wind.cost_per_unit * x_array[:, 1].sum()


def cost_function_vector(x_vector):

    # constraint on conusmer_a
    assert x_vector[:2].sum() == consumer_a.demand
    
    # constraint on conusmer_b
    assert x_vector[2:4].sum() == consumer_b.demand
    
    # constraint on conusmer_c
    assert x_vector[4:].sum() == consumer_c.demand

    # constraint on source_i
    assert x_vector[0::2].sum() <= solar.capacity

    # constraint on source_j
    assert x_array[1::2].sum() <= wind.capacity

    return solar.cost_per_unit * x_vector[0::2].sum() + wind.cost_per_unit * x_vector[1::2].sum()



In [63]:
import numpy as np 

# cols: source, rows: customer

x_array = np.array([
    [1, 4], 
    [2, 8],
    [5, 15],
]) 

x_vector = x_array.flatten()

print(cost_function_array(x_array))
print(cost_function_vector(x_vector))
print(cost_function_array(x_array) == cost_function_vector(x_vector))

2.1500000000000004
2.1500000000000004
True


In [64]:
import numpy as np
xrange = np.arange(0, max(consumer_a.demand, consumer_b.demand, consumer_c.demand), 0.25)
#xrange = [3,4,5,6,7]

In [65]:
from itertools import product

def generate_matrix(xrange):    
    for indices in product(*[xrange] * 6):
        yield np.array(indices)

In [ ]:
xgrid = generate_matrix(xrange)

min_cost = 1e9

for x_vector in generate_matrix(xrange):

    if x_vector[0:2].sum() == consumer_a.demand and \
       x_vector[2:4].sum() == consumer_b.demand and \
       x_vector[4:6].sum() == consumer_c.demand and \
       x_vector[0::2].sum() <= solar.capacity and \
       x_vector[1::2].sum() <= wind.capacity:
        
        cost = cost_function_vector(x_vector)
        
        if cost <= min_cost:
            min_cost = cost 
            print(x_vector, min_cost)

In [ ]:
from scipy.optimize import minimize

max_capacity = max( solar.capacity, wind.capacity )
x_vector = x_array.flatten()

print(x_vector)

res = minimize(
    #cost_function_vector,
    lambda x: solar.cost_per_unit * x_vector[0::2].sum() + wind.cost_per_unit * x_vector[1::2].sum(), #what we want to minimize
    x_vector, 
    constraints = (
        {'type':'ineq','fun': lambda x: solar.capacity - x_vector[0::2].sum() }, # sum first column, need to satisfy solar capacity <= 100
        {'type':'ineq','fun': lambda x: wind.capacity - x_vector[1::2].sum() }, # sum second column, need to satisfy wind capacity <= 150
        {'type':'eq','fun': lambda x: x_vector[0:2].sum() - consumer_a.demand }, # sum first row, need to satisfy Custumer's A need
        {'type':'eq','fun': lambda x: x_vector[2:4].sum() - consumer_b.demand }, # sum second row, need to satisfy Custumer's B need
        {'type':'eq','fun': lambda x: x_vector[4:6].sum() - consumer_c.demand }  # sum third row, need to satisfy Custumer's C need
    ),
    bounds = ((0,max_capacity),(0,max_capacity),(0,max_capacity),(0,max_capacity),(0,max_capacity),(0,max_capacity)),
    method='SLSQP',options={'disp': True,'maxiter' : 100000}
)

res

In [23]:
print(res)
print(res.fun)

True

### TensorFlow Implementation

We will use TensorFlow to approximate a solution via a gradient descent approach, treating constraints as penalty terms in the loss function.

In [55]:
import tensorflow as tf

# Initialize variables
x_solar_A = tf.Variable(1.0, trainable=True)  
x_solar_B = tf.Variable(2.0, trainable=True)
x_solar_C = tf.Variable(5.0, trainable=True)
x_wind_A = tf.Variable(4.0, trainable=True)
x_wind_B = tf.Variable(3.0, trainable=True)
x_wind_C = tf.Variable(0.0, trainable=True)

# Define constraints as penalties
def capacity_constraints():
    return [
        solar.capacity - (x_solar_A + x_solar_B + x_solar_C),  # Solar capacity
        wind.capacity - (x_wind_A + x_wind_B + x_wind_C),    # Wind capacity
        x_solar_A + x_wind_A - consumer_a.demand, # Demand of Consumer A
        x_solar_B + x_wind_B - consumer_b.demand, # Demand of Consumer B
        x_solar_C + x_wind_C - consumer_c.demand  # Demand of Consumer C
    ]

# Define sources, consumers, and their capacities and demands here

# Cost function
cost = solar.cost_per_unit * ( x_solar_A + x_solar_B + x_solar_C ) + wind.cost_per_unit * ( x_wind_A + x_wind_B + x_wind_C )

# Optimizer
optimizer = tf.optimizers.SGD(learning_rate=0.01)
optimizer = tf.optimizers.SGD(learning_rate=0.001)  # Adjust learning rate here

# Training step
def train_step():
    with tf.GradientTape() as tape:
        constraints = capacity_constraints()
        penalty = tf.reduce_sum(tf.square(constraints))  # Squared penalties for constraint violation
        loss = cost + penalty
    gradients = tape.gradient(loss, [x_solar_A, x_solar_B, x_solar_C, x_wind_A, x_wind_B, x_wind_C])
    optimizer.apply_gradients(zip(gradients, [x_solar_A, x_solar_B, x_solar_C, x_wind_A, x_wind_B, x_wind_C]))
    return loss  # Return the loss value

# Optimization loop
for i in range(1000):
    loss = train_step()
    if (i + 1) % 100 == 0:  # Print cost every 100 iterations
        print(f"Iteration {i + 1}, Cost: {loss.numpy()}")


Iteration 100, Cost: 13705.9423828125
Iteration 200, Cost: 11468.130859375
Iteration 300, Cost: 11122.380859375
Iteration 400, Cost: 11062.2734375
Iteration 500, Cost: 11050.0849609375
Iteration 600, Cost: 11047.2099609375
Iteration 700, Cost: 11046.4521484375
Iteration 800, Cost: 11046.2392578125
Iteration 900, Cost: 11046.17578125
Iteration 1000, Cost: 11046.158203125


In [46]:
# Output the results
print(f'           |   Solar            |  Wind')
print(f'Consumer A |{x_solar_A.numpy()}  | {x_wind_A.numpy()}')
print(f'Consumer B |{x_solar_B.numpy()}  | {x_wind_B.numpy()}')
print(f'Consumer C |{x_solar_C.numpy()}  | {x_wind_C.numpy()}')


           |   Solar            |  Wind
Consumer A |20.166656494140625  | 31.833311080932617
Consumer B |17.666685104370117  | 34.33330535888672
Consumer C |15.166643142700195  | 36.83339309692383


In [54]:
loss.numpy()

11046.151